<a href="https://colab.research.google.com/github/Miyeon-Pyeon/bigdataanalysis_practice/blob/master/%EB%B9%85%EB%B6%84%EA%B8%B0_%EC%9E%91%EC%97%852%EC%9C%A0%ED%98%95(%EC%A0%95%EC%8B%9C_%EB%B0%B0%EC%86%A1_%EC%97%AC%EB%B6%80_%ED%8C%90%EB%8B%A8_%EB%8D%B0%EC%9D%B4%ED%84%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv")


display(x_train.head())
display(y_train.head())

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,6045,A,Flight,4,3,266,5,high,F,5,1590
1,44,F,Ship,3,1,174,2,low,M,44,1556
2,7940,F,Road,4,1,154,10,high,M,10,5674
3,1596,F,Ship,4,3,158,3,medium,F,27,1207
4,4395,A,Flight,5,3,175,3,low,M,7,4833


,ID,Reached.on.Time_Y.N
0,6045,0
1,44,1
2,7940,1
3,1596,1
4,4395,1


In [3]:
print(x_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6598 entries, 0 to 6597
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   6598 non-null   int64 
 1   Warehouse_block      6598 non-null   object
 2   Mode_of_Shipment     6598 non-null   object
 3   Customer_care_calls  6598 non-null   object
 4   Customer_rating      6598 non-null   int64 
 5   Cost_of_the_Product  6598 non-null   int64 
 6   Prior_purchases      6598 non-null   int64 
 7   Product_importance   6598 non-null   object
 8   Gender               6598 non-null   object
 9   Discount_offered     6598 non-null   int64 
 10  Weight_in_gms        6598 non-null   int64 
dtypes: int64(6), object(5)
memory usage: 567.1+ KB
None


In [4]:
print(x_train.nunique())

ID                     6598
Warehouse_block           5
Mode_of_Shipment          3
Customer_care_calls       6
Customer_rating           5
Cost_of_the_Product     215
Prior_purchases           8
Product_importance        3
Gender                    2
Discount_offered         65
Weight_in_gms          3365
dtype: int64


In [5]:
print(x_train.isna().sum())

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64


In [6]:
# 사용하지 않은 열 제거

drop_col = ['ID']
x_train_drop = x_train.drop(columns=drop_col)
x_test_drop = x_test.drop(columns=drop_col)

In [7]:
# 모델 구축

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
x_train_dummies = pd.get_dummies(x_train_drop)
y = y_train['Reached.on.Time_Y.N']

In [8]:
x_test_dummies = pd.get_dummies(x_test_drop)
x_test_dummies = x_test_dummies[x_train_dummies.columns]

In [9]:
X_train, X_validation, Y_train, Y_validation = train_test_split(x_train_dummies, y, test_size = 0.33, random_state= 42)
rf = RandomForestClassifier(random_state = 23)
rf.fit(X_train, Y_train)

RandomForestClassifier(random_state=23)

In [10]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

In [12]:
predict_train_label = rf.predict(X_train)
predict_train_proba = rf.predict_proba(X_train)[:,1]

predict_validation_label = rf.predict(X_validation)
predict_validation_proba = rf.predict_proba(X_validation)[:,1]

In [13]:
# accuracy

print('train accuray : ', accuracy_score(Y_train, predict_train_label))
print('validation accuracy : ', accuracy_score(Y_validation, predict_validation_label))

train accuray :  1.0
validation accuracy :  0.6395775941230487


In [14]:
# f1-score

print('train f1-score : ', f1_score(Y_train, predict_train_label))
print('validation f1-score : ', f1_score(Y_validation, predict_validation_label))

train f1-score :  1.0
validation f1-score :  0.6744089589382


In [16]:
# recall

print('train recall score : ', recall_score(Y_train, predict_train_label))
print('validation recall score : ', recall_score(Y_validation, predict_validation_label))

train recall score :  1.0
validation recall score :  0.630721489526765


In [17]:
# precision

print('train precision score : ', precision_score(Y_train, predict_train_label))
print('validation precision score : ', precision_score(Y_validation, predict_validation_label))

train precision score :  1.0
validation precision score :  0.7245989304812834


In [19]:
# auc

print('train auc score : ', roc_auc_score(Y_train, predict_train_label))
print('validation auc score : ', roc_auc_score(Y_validation, predict_validation_proba))

train auc score :  1.0
validation auc score :  0.7261997118475008


In [20]:
predict_test_label = rf.predict(x_test_dummies)
predict_test_proba = rf.predict_proba(x_test_dummies)[:,1]

In [21]:
pd.DataFrame({'ID':x_test.ID, 'Reached.on.Time':predict_test_label}).to_csv('00300.csv', index=False)